### Photo to Sketch: Your Artificial Street Artist in the Cloud

**Photo to Sketch: Your Artificial Street Artist in the Cloud**, showcase an ML use case that take pictures in an ios application and generate their sketched version. To accomplish it, hosts a pre-trained Fast Arbitrary image style transfer model from TensorFlow Hub in Amazon SageMaker. For details on the ML model,you can check the following website: https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2

The goal of this Jupyter Notebook, is to host the pre-trained model in an Amazon SageMaker Endpoint and check if inference works with a sample image. This Notebook can be executed using the *conda_python3* Kernel in an Amazon SageMaker Notebook instance.

#### 0. Amazon SageMaker Execution Role

Before running the notebook, check that your Amazon SageMaker IAM role have the necessary permissions.

- arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess
- arn:aws:iam::aws:policy/AmazonS3FullAccess
- arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
- arn:aws:iam::aws:policy/AmazonSageMakerPipelinesIntegrations
- arn:aws:iam::aws:policy/AWSLambda_FullAccess
- arn:aws:iam::aws:policy/AWSCodePipeline_FullAccess


#### 1. Import libraries and install dependencies

In [ ]:
import boto3
import datetime
import base64
from sagemaker import get_execution_role
import json

In [ ]:
from PIL import Image
import numpy as np
from io import BytesIO
import json
import io

In [ ]:
import os
import boto3
import sagemaker
import sagemaker.session
import datetime
import json
import time

#Import Amazon SageMaker pipelines
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
)
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.workflow.step_collections import RegisterModel

from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (LambdaStep,LambdaOutput,LambdaOutputTypeEnum,)

In [ ]:
#Import create_lambda_role function from lambda/iam_helper.py
import sys
sys.path.insert(0, "./lambda")
import iam_helper
from iam_helper import create_lambda_role

#### 2. Variables

Define the necessary parameters to run this notebook.

In [ ]:
#Session variables
role = get_execution_role()
sm = boto3.client("sagemaker")
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

In [ ]:
#s3 bucket
bucket = "photo-to-sketch-{}".format(account_id)

In [ ]:
#Repository name for the custom image for the endpoint.  
image_repo_name = "photo-to-sketch-byoc-tensorflow"
ts = datetime.datetime.now().strftime('%m%d-%H%M')
endpoint_name = "style-transfer-{}".format(ts)
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account_id, region, image_repo_name)

In [ ]:
#Amazon S3 model path
model_url = f"s3://{bucket}/model/magenta_arbitrary-image-stylization-v1-256_2.tar.gz"

#### 3. Host Pre-trained ML model to Amazon SageMaker Endpoint for real-time inference

To host the pre-trained Tensorflow model specified above, we will be creating our custom container for inference. 

##### 3.1 Create ECR repository to store the custom image

In [ ]:
try:
    ecr = boto3.client('ecr')
    image_repo = ecr.create_repository(repositoryName = image_repo_name)   
except Exception as e:
    print("Error: {}".format(e))

##### 3.2 Locally build the docker image and push it to ECR

In [ ]:
!pip install sagemaker-studio-image-build

In [ ]:
%%bash

chmod +x src/serve

sm-docker build . --repository photo-to-sketch-byoc-tensorflow:latest

##### 3.3 Create Amazon SageMaker Model & Endpoint via Amazon SageMaker Pipelines

To deploy the ML model into an Amazon SageMaker endpoint, we will create a deployment pipeline by leveraging Amazon SageMaker Pipelines. Find below the different steps.

##### Pipeline variables

In [ ]:
pipeline_name="style-transfer-pipeline-tf"
base_job_prefix="style-transfer-tf-"

##### Create Amazon SageMaker model step

The first step in our pipeline will create the Amazon SageMaker model. It will be executed in a custom Lambda step. 

In [ ]:
%%writefile ./lambda/create_model.py

"""
    This Lambda function creates a SageMaker model.
    As input event, it receives the endpoint_name, the image_uri and the execution role. 
"""

import json
import boto3 
     
def lambda_handler(event, context):   
    
    #Amazon SageMaker session
    sm = boto3.client("sagemaker")
    region = boto3.Session().region_name
    
    #Input parameters
    endpoint_name = event['endpoint_name']
    image_uri = event['image_uri']
    role = event['role']
    model_url = event['model_path']
    
    #Create a Model using Amazon SageMaker 
    model = sm.create_model(
        ModelName=endpoint_name,
            Containers=[
                {
                    "Image": image_uri,
                    'Mode': 'SingleModel',
                    'ModelDataUrl': model_url,
                },
            ],
            ExecutionRoleArn=role,
            EnableNetworkIsolation=False,
    )
    
    return {
        "statusCode": 200,
        "body": json.dumps("Created Model!"),
        "model_name": str(endpoint_name),
    }


In [ ]:
#Create AWS Lambda role for custom step
lambda_role = create_lambda_role("lambda-deployment-role")
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
function_name = "sagemaker-lambda-step-sagemaker-model-" + current_time

In [ ]:
# Create Lambda function
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="./lambda/create_model.py",
    handler="create_model.lambda_handler",
)

In [ ]:
# Output parameters for Lambda
output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="model_name", output_type=LambdaOutputTypeEnum.String)

In [ ]:
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{image_repo_name}:latest'

step_create_model_lambda = LambdaStep(
    name="LambdaStepModelCreate",
    lambda_func=func,
    inputs={
        "endpoint_name": endpoint_name,
        "image_uri": image_uri,
        "role": lambda_role,
        "model_path": model_url
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

##### Create Amazon SageMaker Endpoint & Endpoint config step

In [ ]:
%%writefile ./lambda/create_endpoint.py

"""
    This Lambda function creates an Endpoint Configuration and deploys a model to an Endpoint. 
    The name of the model to deploy is provided via the event argument.
    The Lambda also saves the endpoint_name in Parameter Store.
"""

import json
import boto3 
import time
    
def lambda_handler(event, context):    
    
    #Amazon SageMaker session
    sm = boto3.client("sagemaker")
    region = boto3.Session().region_name
    endpoint_name = event["endpoint_name"]
    
    time.sleep(10)
    
    #Create Endpoint Configuration & endpoint in a Lambda
    endpoint_config = sm.create_endpoint_config(
        EndpointConfigName=endpoint_name,
        ProductionVariants=[
            {
                'VariantName': endpoint_name,
                'ModelName': endpoint_name,
                'InitialInstanceCount': 1,
                'InstanceType': 'ml.m4.xlarge',
            }
        ]
    )
        
    #Create Endpoint
    endpoint = sm.create_endpoint(
        EndpointName=endpoint_name,
        EndpointConfigName=endpoint_name
    )
    
    #Register endpoint name to Parameter Store
    ssm = boto3.client('ssm')
    ssm.put_parameter(Name='endpoint_name',Value=endpoint_name,Type='String',Overwrite=True)
    
    return {
        "statusCode": 200,
        "body": json.dumps("Created Endpoint!")
    }

In [ ]:
#Create AWS Lambda role for custom step
lambda_role = create_lambda_role("lambda-deployment-role")
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
function_name = "sagemaker-lambda-endpoint-step-sagemaker-model-" + current_time

In [ ]:
# Create Lambda function
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="./lambda/create_endpoint.py",
    handler="create_endpoint.lambda_handler",
)

In [ ]:
# Output parameters for Lambda
output_param_4 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_5 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)

In [ ]:
step_create_endpoint_lambda = LambdaStep(
    name="LambdaStepEndpointCreate",
    lambda_func=func,
    inputs={
        "endpoint_name": endpoint_name,
    },
    outputs=[output_param_4, output_param_5],
)

##### Define Amazon SageMaker pipeline 

In [ ]:
sm_client = boto3.client("sagemaker")
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.session.Session(boto_session=boto_session, sagemaker_client=sm_client)

# pipeline instance
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[],
    steps=[step_create_model_lambda,step_create_endpoint_lambda],
    sagemaker_session=sagemaker_session,
)

##### Execute the pipeline

In [ ]:
definition = json.loads(pipeline.definition())
print(definition)

In [ ]:
pipeline.upsert(role_arn = role)

In [ ]:
lambda_role = create_lambda_role("lambda-deployment-role")

In [ ]:
execution = pipeline.start()

In [ ]:
execution.wait()

##### Wait until endpoint is provisioned

In [ ]:
response = sm.describe_endpoint(EndpointName = endpoint_name)
print("Endpoint {} status: {}".format(endpoint_name,response['EndpointStatus']))

In [ ]:
while response['EndpointStatus'] == 'Creating':
    response = sm.describe_endpoint(EndpointName = endpoint_name)
    time.sleep(10)
    print("Endpoint creating...")
print("Endpoint created!")

#### 4. Inference 

Now that we have the Amazon SageMaker Endpoint provisioned with our ML pre-trained model, we are ready to test inference in our model!
To do so, we will leverage the AWS Lambda previously deployed by our CFN template!

Our AWS Lambda is a container-based image, so we will build the image and push it there. 

##### 4.0 Create ECR repository for hosting Lambda image

In [ ]:
image_repo_name = "photo-to-sketch-lambda-tensorflow"
try:
    ecr = boto3.client('ecr')
    image_repo = ecr.create_repository(repositoryName = image_repo_name)   
except Exception as e:
    print("Error: {}".format(e))

##### 4.1 Create Lambda image and push it to ECR

In [ ]:
%%bash

cd lambda-inference

sm-docker build --repository photo-to-sketch-lambda-tensorflow:latest . 

##### 4.2 Create Lambda with our new deployed image

In [ ]:
lambda_func_name = f"photo-to-sketch-inference-lambda-{account_id}"
latest_image = f"{account_id}.dkr.ecr.{region}.amazonaws.com/photo-to-sketch-lambda-tensorflow:latest"
print(latest_image)

In [ ]:
# Create container-based Lambda function
!aws lambda create-function --region $region --function-name $lambda_func_name \
    --package-type Image  \
    --code ImageUri=$latest_image   \
    --role $lambda_role \
    --memory-size 1000 \
    --timeout 300

In [ ]:
#latest image pushed in the ECR repository
print(f"My latest image for Lambda pushed to ECR: {latest_image}")

In [ ]:
#In case you would like to update Lambda's image:
#!aws lambda update-function-code --function-name $lambda_func_name --image-uri $latest_image

##### 4.3 Test inference with a sample image!

**Note: Wait until Lambda has been updated. You can check it in the AWS Console**

Now, to be able to test how our model performs, we provided one sample image from a dog. You can find the original image under /img folder.  

##### Encode image as base64

Our ML model expects the image as a base64 encoded image. Therefore, we will visualize the original image and encode it as base64. 

In [ ]:
#Open image using PIL
from PIL import Image
im = Image.open("./img/dog.jpeg")
rgb_im = im.convert('RGB')
rgb_im.save("./img/dog.jpeg")
#Encode the image as base64
encoded = base64.b64encode(open("./img/dog.jpeg", "rb").read()).decode("utf-8")

In [ ]:
#Visualize the original sample image
display(rgb_im)

##### Decide effectStyle

We have 4 different styles that you can select from different painters, select the style you want the most! Find below the styles

In [ ]:
from IPython.display import Image
print("Style 1: \n")
Image('./img/style/1.jpeg', width=200)

In [ ]:
print("Style 2: \n")
Image('./img/style/2.jpeg', width=200)

In [ ]:
print("Style 3: \n")
Image('./img/style/3.jpeg', width=200)

In [ ]:
print("Style 4: \n")
Image('./img/style/4.jpeg', width=200)

##### Invoke Lambda with the selected effectStyle

Invoke the AWS Lambda function 4 times and get the styled dog image with the 4 different styles!

Our AWS Lambda will be later called by API Gateway, therefore, the payload should have the same format.

In [ ]:
from PIL import Image
encoded_images = []
for effect_style in [1,2,3,4]:
    #Lambda event - encoded image and effect style selected
    payload =  json.dumps({'body':{'image' : str(encoded), 'effectType' : str(effect_style)}}).encode("utf-8")
    #Invoke Lambda
    client = boto3.client('lambda')
    response = client.invoke(FunctionName=lambda_func_name,InvocationType='RequestResponse',Payload=payload)
    #print(json.loads(response['Payload'].read().decode()))
    payload = json.loads(response['Payload'].read().decode())['body']
    style_image_encoded = json.loads(payload)['image']
    #Save base64 encoded image
    encoded_images.append(style_image_encoded)
    #Decode from base64 to PIL image
    image = base64.b64decode(str(style_image_encoded))
    img = Image.open(io.BytesIO(image))
    img.save("./img/dog-styled-{}.jpeg".format(effect_style), 'jpeg')
    print("Image styled {} saved!".format(effect_style))

##### Display the styled images

See below your dog image styled with the 4 different paintings!

In [ ]:
print("Style 1: \n")
image = base64.b64decode(str(encoded_images[0]))       
img = Image.open(io.BytesIO(image))
display(img)

In [ ]:
print("Style 2: \n")
image = base64.b64decode(str(encoded_images[1]))       
img = Image.open(io.BytesIO(image))
display(img)

In [ ]:
print("Style 3: \n")
image = base64.b64decode(str(encoded_images[2]))       
img = Image.open(io.BytesIO(image))
display(img)

In [ ]:
print("Style 4: \n")
image = base64.b64decode(str(encoded_images[3]))       
img = Image.open(io.BytesIO(image))
display(img)

#### 5. Delete endpoint

Delete the Amazon SageMaker endpoint when you are done with inference the model! :) 

In [ ]:
print(sm.delete_endpoint(EndpointName = endpoint_name))